Install PyGithub because it doesn't exist in the docker container by default.

In [ ]:
import sys
!{sys.executable} -m pip install PyGithub

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf

from issue_tagging_bot.issue_data import Stage1PreprocData, Stage2PreprocData

In [3]:
stage2 = Stage2PreprocData()

In [4]:
stage2.stage1.top_n_topics()

6.topic: TeX                      48
6.topic: reproducible builds      49
6.topic: rust                     51
6.topic: xfce                     59
6.topic: steam                    68
6.topic: user experience          73
6.topic: closure-size            114
6.topic: kernel                  115
6.topic: cross-compilation       132
6.topic: gnome3                  263
6.topic: qt/kde                  345
6.topic: haskell                 606
6.topic: darwin                  610
6.topic: python                  813
6.topic: nixos                  1169
dtype: int64

In [5]:
train_set, val_set, test_set = stage2.to_datasets()

In [14]:
all_test_labels = []
all_nums = []
for issue_body, labels, num in test_set:
    all_test_labels.append(labels)
    all_nums.append(num)
    
first_all_test_labels = all_test_labels[0]
first_all_nums = all_nums[0]

first_all_test_labels, first_all_nums

(<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=79304>)

In [7]:
all_test_labels_tensor = tf.reshape(tf.concat(all_test_labels, axis=0), (-1, stage2.stage1.top_n_topics().shape[0]))

In [8]:
np.sum(all_test_labels_tensor.numpy(), axis=0)

array([  4,   3,   6,  11,   3,  10,   7,  17,  22,  33,  32,  56,  60,
        85, 102])

In [10]:
all_test_labels_tensor[0]

<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])>

In [12]:
all_nums[0]

<tf.Tensor: shape=(), dtype=int32, numpy=79304>

In [65]:
only_nixos_labels = test_set.map(lambda issue_body, labels, issue_num: (issue_body, labels[-1], issue_num))

In [72]:
list(only_nixos_labels.as_numpy_iterator())[:4]

[(1, 79304), (0, 10809), (0, 66303), (0, 21297)]

In [81]:
np.sum(np.array(list(only_nixos_labels.as_numpy_iterator())), axis=0)

array([     102, 56734384])